# Regress QAP measures against scanning parameters

In [1]:
# Load some needed packages
library(grid)
library(gridExtra)
library(ggplot2)
library(dplyr)
library(repr)
library(reshape2)
library(tidyr)
library(lme4)


Attaching package: ‘dplyr’

The following object is masked from ‘package:gridExtra’:

    combine

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘tidyr’

The following object is masked from ‘package:reshape2’:

    smiths

Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand



## Read in and tidy the sMRI scanning parameters for ABIDE

In [2]:
abide_smri_scan_params<-read.csv("anat_abide_scan_params_bids.csv")

abide_smri_scan_params<-abide_smri_scan_params %>% 
                     mutate(Scanner = paste(Manufacturer,ManufacturersModelName))
abide_smri_scan_params$Scanner<-factor(abide_smri_scan_params$Scanner)

abide_smri_scan_params<-abide_smri_scan_params %>% 
                     rowwise() %>% 
                       mutate(VoxelVolume = as.numeric(SliceThickness)*
                                            prod(as.numeric(unlist(strsplit(as.character(PixelSpacing), "x")))))

abide_smri_scan_params<-abide_smri_scan_params %>% 
                     select(Site,Scanner,EchoTime,RepetitionTime,InversionTime,FlipAngle,VoxelVolume) %>% 
                       drop_na()

abide_smri_scan_params$Scanner<-droplevels(abide_smri_scan_params$Scanner)

## Read in and tidy the sMRI QAP measures for ABIDE

In [3]:
abide_anat_spat_df<-read.csv("2016_05_ABIDE_qap_anatomical_spatial.csv")

id.vars=c('Participant','Site','Session','Series')
measure.vars=c('CNR','Cortical.Contrast','EFC','FBER','FWHM','Qi1','SNR')
abide_anat_spat_df<-abide_anat_spat_df[c(id.vars,measure.vars)]

# reduce to just session_1 and anat_1 and remove rows with missing values
abide_anat_spat_df <- abide_anat_spat_df %>% 
                        filter(Session == "session_1" & Series == "anat_1") %>% 
                           drop_na()

# make sure that participant is a factor
abide_anat_spat_df$Participant <- factor(abide_anat_spat_df$Participant)

# summary(abide_anat_spat_df)

# plots
qap_label_strings=c(CNR='CNR',
                Cortical.Contrast='Cortical Contrast',
                EFC='EFC',
                FBER='FBER',
                FWHM='Smoothness (FWHM)',
                Qi1='Fraction of Artifact Voxels',
                SNR='SNR')

## Analyze!

In [5]:
abide_anat_df <- inner_join(abide_anat_spat_df, abide_smri_scan_params, by="Site")

# mean center all of the covariates
abide_anat_df<-abide_anat_df %>% mutate_if(is.numeric, funs(scale(., scale=FALSE)))



In [15]:
abide_smri_qap_param_model<-lm(FWHM ~ 1 + Scanner + EchoTime + InversionTime + 
                               RepetitionTime + FlipAngle + VoxelVolume,
                               abide_anat_df)
summary(abide_smri_qap_param_model)


Call:
lm(formula = FWHM ~ 1 + Scanner + EchoTime + InversionTime + 
    RepetitionTime + FlipAngle + VoxelVolume, data = abide_anat_df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.26108 -0.34565  0.00362  0.32172  2.10075 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)              0.19592    0.11752   1.667 0.095852 .  
ScannerPhilips Intera   -0.57989    0.10621  -5.460 6.22e-08 ***
ScannerSiemens Allegra  -0.74460    0.13527  -5.505 4.87e-08 ***
ScannerSiemens TrioTim   0.49992    0.17678   2.828 0.004792 ** 
ScannerSiemens Verio    -0.86688    0.15884  -5.457 6.30e-08 ***
EchoTime               219.55242   57.77197   3.800 0.000155 ***
InversionTime            0.67773    0.36854   1.839 0.066258 .  
RepetitionTime          -0.92471    0.08747 -10.571  < 2e-16 ***
FlipAngle               -0.53287    0.07341  -7.259 8.64e-13 ***
VoxelVolume             -0.65736    0.19646  -3.346 0.000855 ***
---
Signif. codes:  0 ‘***’ 0